In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w5-p1/train.csv
/kaggle/input/2021-ai-w5-p1/test.csv
/kaggle/input/2021-ai-w5-p1/sample_submit.csv


In [2]:
train = pd.read_csv("/kaggle/input/2021-ai-w5-p1/train.csv")
test = pd.read_csv("/kaggle/input/2021-ai-w5-p1/test.csv")
submit = pd.read_csv("/kaggle/input/2021-ai-w5-p1/sample_submit.csv")
print(train.info())
print()
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to 28x28
dtypes: int64(785)
memory usage: 359.3 MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 784 entries, 1x1 to 28x28
dtypes: int64(784)
memory usage: 59.8 MB
None


In [3]:
train.head(5)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = train.drop('label', axis = 1)
y = train['label']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state = 1, stratify = y)

In [6]:
import torch
import torch.optim as optim
import torch.nn.functional as F

torch.manual_seed(1)

In [7]:
X_train = torch.FloatTensor(X_train.to_numpy())
X_val = torch.FloatTensor(X_val.to_numpy())
y_train = torch.LongTensor(y_train.to_numpy())
y_val = torch.LongTensor(y_val.to_numpy())

X_test = torch.FloatTensor(test.to_numpy())

In [8]:
# import torch.cuda
# print(torch.cuda.is_available())
# torch.cuda.init()
# torch.cuda.manual_seed(1)

# X_train.cuda()
# y_train.cuda()
# X_val.cuda()
# y_val.cuda()
# X_test.cuda()

In [9]:
X_train.shape

torch.Size([42000, 784])

In [10]:
y.nunique()

10

In [11]:
# 1. learnig rate
nb_epoch = 1000
lrs = [0.1, 0.01,0.001,0.0001]

for lr in lrs:
    W = torch.zeros((784,10), requires_grad = True)
    b = torch.zeros(1, requires_grad = True)
    optimizer = optim.SGD([W,b], lr = lr)
    for epoch in range(nb_epoch + 1):
        H = torch.softmax(X_train@W + b, dim = 1)   
        cost = F.cross_entropy(X_train@W + b, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if epoch % 100 == 0 :
            print('lr:{}, epoch:{}, cost:{}'.format(lr, epoch, cost.item()))
            
    pred = torch.argmax(torch.softmax(X_train@W + b, dim = 1), dim=1)
    correct_pred = pred.float() == y_train
    train_acc = correct_pred.sum().item() / len(correct_pred) * 100

    pred = torch.argmax(torch.softmax(X_val@W + b,dim=1), dim=1)
    correct_pred = pred.float() == y_val
    val_acc = correct_pred.sum().item() / len(correct_pred) * 100

    print('lr:{}, train_acc:{}, val_acc:{}'.format(lr, train_acc, val_acc), end = '\n\n\n\n')

lr:0.1, epoch:0, cost:2.3026933670043945
lr:0.1, epoch:100, cost:1506.9742431640625
lr:0.1, epoch:200, cost:3357.922607421875
lr:0.1, epoch:300, cost:420.6086730957031
lr:0.1, epoch:400, cost:1707.70703125
lr:0.1, epoch:500, cost:474.9587097167969
lr:0.1, epoch:600, cost:398.2598571777344
lr:0.1, epoch:700, cost:467.3810119628906
lr:0.1, epoch:800, cost:1103.6129150390625
lr:0.1, epoch:900, cost:866.5687866210938
lr:0.1, epoch:1000, cost:347.08905029296875
lr:0.1, train_acc:92.28333333333333, val_acc:90.87222222222222



lr:0.01, epoch:0, cost:2.3026933670043945
lr:0.01, epoch:100, cost:297.418701171875
lr:0.01, epoch:200, cost:54.85395812988281
lr:0.01, epoch:300, cost:45.57455825805664
lr:0.01, epoch:400, cost:45.766944885253906
lr:0.01, epoch:500, cost:85.80814361572266
lr:0.01, epoch:600, cost:50.542232513427734
lr:0.01, epoch:700, cost:48.0660285949707
lr:0.01, epoch:800, cost:42.271278381347656
lr:0.01, epoch:900, cost:37.01537322998047
lr:0.01, epoch:1000, cost:651.0580444335938

In [12]:
W = torch.zeros((784,10), requires_grad = True)
b = torch.zeros(1, requires_grad = True)
optimizer = optim.SGD([W,b], lr = 0.1)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)

for epoch in range(nb_epoch + 1):
    H = torch.softmax(X_train@W + b, dim = 1)   
    cost = F.cross_entropy(X_train@W + b, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    scheduler.step()

    
    if epoch % 100 == 0 :
        print('lr:{}, epoch:{}, cost:{}'.format(scheduler.get_last_lr(), epoch, cost.item()))

pred = torch.argmax(torch.softmax(X_train@W + b, dim = 1) , dim=1)
correct_pred = pred.float() == y_train
train_acc = correct_pred.sum().item() / len(correct_pred) * 100

pred = torch.argmax(torch.softmax(X_val@W + b,dim=1), dim=1)
correct_pred = pred.float() == y_val
val_acc = correct_pred.sum().item() / len(correct_pred) * 100
print('lr:{}, train_acc:{}, val_acc:{}'.format('StepLR', train_acc, val_acc), end = '\n\n\n\n')

lr:[0.1], epoch:0, cost:2.3026933670043945
lr:[0.05], epoch:100, cost:1506.9742431640625
lr:[0.025], epoch:200, cost:326.66815185546875
lr:[0.0125], epoch:300, cost:259.806396484375
lr:[0.00625], epoch:400, cost:233.75823974609375
lr:[0.003125], epoch:500, cost:222.1692352294922
lr:[0.0015625], epoch:600, cost:216.70199584960938
lr:[0.00078125], epoch:700, cost:214.04006958007812
lr:[0.000390625], epoch:800, cost:212.72386169433594
lr:[0.0001953125], epoch:900, cost:212.072021484375
lr:[9.765625e-05], epoch:1000, cost:211.74655151367188
lr:StepLR, train_acc:92.09285714285714, val_acc:91.08333333333334





In [13]:
# 1. learnig rate 결과 : StepLR을 사용했을 때 test와 가장 비슷하다
nb_epoch = 1000
W = torch.zeros((784,10), requires_grad = True)
b = torch.zeros(1, requires_grad = True)
optimizer = optim.SGD([W,b], lr = 0.1)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)

for epoch in range(nb_epoch + 1):
    H = torch.softmax(X_train@W + b, dim = 1)   
    cost = F.cross_entropy(X_train@W + b, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    scheduler.step()

y_pred = torch.argmax(torch.softmax(X_test@W+b, dim = 1), dim = 1)
submit['label'] = y_pred.detach().numpy()
submit.to_csv('submit_lr.csv', mode = 'w', index = False)
submit

,id,label
0,1,6
1,2,1
2,3,0
3,4,0
4,5,9
...,...,...
9995,9996,3
9996,9997,7
9997,9998,2
9998,9999,8


In [14]:
# 2. batch size

from torch.utils.data import TensorDataset, DataLoader

batch_sizes = [100,1000,10000]
dataset = TensorDataset(X_train, y_train)

for batch_size in batch_sizes:
    
    dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = True)
    W = torch.zeros((784,10), requires_grad = True)
    b = torch.zeros(1, requires_grad = True)
    optimizer = optim.SGD([W,b], lr = 0.01)

    nb_epoch = 100
    for epoch in range(nb_epoch + 1):
        for batch_idx, samples in enumerate(dataloader):
            X_mini, y_mini = samples
            H = torch.softmax(X_mini@W + b, dim = 1)   
            cost = F.cross_entropy(X_mini@W + b, y_mini)

            optimizer.zero_grad()
            cost.backward()
            optimizer.step()
            
    pred = torch.argmax(H, dim=1)
    correct_pred = pred.float() == y_mini
    train_acc = correct_pred.sum().item() / len(correct_pred) * 100

    pred = torch.argmax(torch.softmax(X_val@W + b,dim=1), dim=1)
    correct_pred = pred.float() == y_val
    val_acc = correct_pred.sum().item() / len(correct_pred) * 100

    print('batch_size:{}, train_acc:{}, val_acc:{}'.format(batch_size, train_acc, val_acc))

batch_size:100, train_acc:85.0, val_acc:80.9388888888889
batch_size:1000, train_acc:91.4, val_acc:90.08333333333334
batch_size:10000, train_acc:92.10000000000001, val_acc:91.03333333333333


In [15]:
#2 실험결과 batch size가 100일 때 가장 정확도가 높았음(적용안한 SGD보다는 대체로 낮았음)

dataloader = DataLoader(dataset, batch_size = 100, shuffle = True)
W = torch.zeros((784,10), requires_grad = True)
b = torch.zeros(1, requires_grad = True)
optimizer = optim.SGD([W,b], lr = 0.01)

nb_epoch = 100
for epoch in range(nb_epoch + 1):
    for batch_idx, samples in enumerate(dataloader):
        X_mini, y_mini = samples
        H = torch.softmax(X_mini@W + b, dim = 1)   
        cost = F.cross_entropy(X_mini@W + b, y_mini)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

y_pred = torch.argmax(torch.softmax(X_test@W+b, dim = 1), dim = 1)
submit['label'] = y_pred.detach().numpy()
submit.to_csv('submit_batch.csv', mode = 'w', index = False)
submit        

,id,label
0,1,6
1,2,1
2,3,0
3,4,5
4,5,9
...,...,...
9995,9996,3
9996,9997,7
9997,9998,2
9998,9999,8


In [16]:
# 3. epochs
W = torch.zeros((784,10), requires_grad = True)
b = torch.zeros(1, requires_grad = True)
optimizer = optim.SGD([W,b], lr = 0.0001)

acc_diff = 100
nb_epoch = 10000

for epoch in range(nb_epoch + 1):
    H = torch.softmax(X_train@W + b, dim = 1)   
    cost = F.cross_entropy(X_train@W + b, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if (epoch % 1000) == 0:
        pred = torch.argmax(torch.softmax(X_train@W + b, dim = 1), dim=1)
        correct_pred = pred.float() == y_train
        train_acc = correct_pred.sum().item() / len(correct_pred) * 100
        
        pred = torch.argmax(torch.softmax(X_val@W + b,dim=1), dim=1)
        correct_pred = pred.float() == y_val
        val_acc = correct_pred.sum().item() / len(correct_pred) * 100
        print('epochs:{}, cost:{:.4f} train_acc:{:.2f}, val_acc:{:.2f}'.format(epoch, cost.item(),train_acc, val_acc))

        if acc_diff > abs(train_acc - val_acc) and train_acc > 90:
            acc_diff = abs(train_acc - val_acc) 
            W_save = W.clone()
            b_save = b.clone()
            print('update W, b')
    

epochs:0, cost:2.3027 train_acc:67.21, val_acc:67.58
epochs:1000, cost:0.4316 train_acc:92.75, val_acc:91.44
update W, b
epochs:2000, cost:2.0310 train_acc:87.46, val_acc:86.34
epochs:3000, cost:0.3540 train_acc:93.58, val_acc:91.45
epochs:4000, cost:0.6654 train_acc:90.89, val_acc:89.24
epochs:5000, cost:0.3614 train_acc:93.43, val_acc:91.20
epochs:6000, cost:0.9395 train_acc:88.80, val_acc:87.16
epochs:7000, cost:0.3514 train_acc:93.16, val_acc:90.67
epochs:8000, cost:0.3315 train_acc:93.86, val_acc:91.08
epochs:9000, cost:0.3429 train_acc:93.80, val_acc:91.12
epochs:10000, cost:0.3441 train_acc:93.81, val_acc:91.12


In [17]:
# 3 실험결과 epoch = 1000일 때 train, val 의 정확도가 차이가 가장 작았음. 

y_pred = torch.argmax(torch.softmax(X_test@W_save+b_save, dim = 1), dim = 1)
submit['label'] = y_pred.detach().numpy()
submit.to_csv('submit_epoch.csv', mode = 'w', index = False)
submit

,id,label
0,1,6
1,2,1
2,3,0
3,4,0
4,5,9
...,...,...
9995,9996,3
9996,9997,7
9997,9998,2
9998,9999,8
